In [2]:
import requests
import base64
from mimetypes import guess_type

class GPT4VClient:
    def __init__(self, api_key):
        self.api_key = api_key
        self.headers = {
            "Content-Type": "application/json",
            "api-key": api_key,
        }
        self.endpoint = "https://anankesweden.openai.azure.com/openai/deployments/AnankeVision/chat/completions?api-version=2023-07-01-preview"

    def request_with_url_image(self, image_url, prompt):
        payload = self._build_payload_with_image_url(image_url, prompt)
        return self._send_request(payload)

    def request_with_local_image(self, image_path, prompt):
        payload = self._build_payload_with_local_image(image_path, prompt)
        return self._send_request(payload)

    def _build_payload_with_image_url(self, image_url, prompt):
        payload = {
            "messages": [
                {
                    "role": "system",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": image_url
                            }
                        }
                    ]
                }
            ],
            "temperature": 0.7,
            "top_p": 0.95,
            "max_tokens": 800
        }
        return payload

    def _build_payload_with_local_image(self, image_path, prompt):
        mime_type, _ = guess_type(image_path)
        if mime_type is None:
            mime_type = 'application/octet-stream'

        with open(image_path, "rb") as image_file:
            base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

        payload = {
            "messages": [
                {
                    "role": "system",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:{mime_type};base64,{base64_encoded_data}"
                            }
                        }
                    ]
                }
            ],
            "temperature": 0.7,
            "top_p": 0.95,
            "max_tokens": 800
        }
        return payload

    def _send_request(self, payload):
        try:
            response = requests.post(self.endpoint, headers=self.headers, json=payload)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            raise SystemExit(f"Failed to make the request. Error: {e}")

# Example usage:
api_key = "45d24fc4b19047eaa81632eb76dc0fa9"
client = GPT4VClient(api_key)
# image_url_response = client.request_with_url_image("https://miro.medium.com/max/1400/1*BHzGVskWGS_3jEcYYi6miQ.png", "这张图在讲什么.")
# print("Response from URL image:", image_url_response)
local_image_response = client.request_with_local_image("/workspace/tanwnexuan/ananke/example/pipeline/ret.png", "这张图和transformer有什么区别.")
print("Response from local image:", local_image_response)


Response from local image: {'id': 'chatcmpl-8xbm3rQ0E2XkU9K3g91GBpV4h07Cl', 'object': 'chat.completion', 'created': 1709217247, 'model': 'gpt-4', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '这张图展示了RetNet的两种表示形式：并行表示和循环表示。RetNet是一种注意力机制，它将传统的Transformer中的自注意力（self-attention）模块替换为一个轻量级的递归模块。在这张图中，我们可以看到两种不同的表示形式，它们都具有Q（查询）、K（键）、V（值）这三个主要组件，这些组件是Transformer中自注意力机制的基础。\n\n(a) 部分显示的是并行表示，其中输入X被分解为Q、K、V三个部分，然后通过点积和按元素乘法（表示为∘）来计算注意力权重，最后将这些权重应用到V上得到输出O。输出O在通过GroupNorm（表示为GN）标准化后得到最终结果。\n\n(b) 部分则展示了循环表示，其中引入了一个循环状态S，它在不同的时间步之间传递信息。每个时间步n，输入Xn更新循环状态Sn，并且使用循环状态来计算新的Qn、Kn和Vn。这些组件通过相似的注意力机制来更新状态，并最终产生输出On。\n\n与标准的Transformer相比，这里的主要区别在于RetNet使用了循环表示来处理序列数据，而不是标准Transfor

![](./ret.png)

In [3]:
import os
from openai import AzureOpenAI


client = AzureOpenAI(
  azure_endpoint = "https://anankeus.openai.azure.com/", 
  api_key="61bc1aab37364618ae0df70bf5f340dd",  
  api_version="2024-02-15-preview"
)


message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."}]

completion = client.chat.completions.create(
  model="Ananke3-1106-US-WEST", # model = "deployment_name"
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)
print(completion)

ChatCompletion(id='chatcmpl-8xbmGalhWoUSzmIga0FYxeMUKuZzF', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, logprobs=None)], created=1709217260, model='gpt-35-turbo', object='chat.completion', usage=CompletionUsage(completion_tokens=9, prompt_tokens=18, total_tokens=27), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], system_fingerprint='fp_68a7d165bf')


In [4]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="6d1e089b49d9a1fc9f251240af39d2ff.sEjOi7HBGjQML5pf") # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {"role": "user", "content": "作为一名营销专家，请为我的产品创作一个吸引人的slogan"},
        {"role": "assistant", "content": "当然，为了创作一个吸引人的slogan，请告诉我一些关于您产品的信息"},
        {"role": "user", "content": "智谱AI开放平台"},
        {"role": "assistant", "content": "智启未来，谱绘无限一智谱AI，让创新触手可及!"},
        {"role": "user", "content": "创造一个更精准、吸引人的slogan"}
    ],
)
print(response.choices[0].message)


content='"精准智绘，未来从此触手可及 —— 智谱AI，赋能想象。"' role='assistant' tool_calls=None


In [5]:
print(response.choices)

[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='"精准智绘，未来从此触手可及 —— 智谱AI，赋能想象。"', role='assistant', tool_calls=None))]
